#### Import the essential packages that will be used.

In [1]:
import numpy as np
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras import backend as K
from keras.preprocessing import image
import matplotlib.pyplot as plt
import scipy
import seaborn as sns
import pickle

#### Directory and define the labels/features

In [2]:
# set the image dimenions, all images will have height & width set to 200x200
img_width, img_height = 200, 200

# directory
train_data_dir = 'organic_and_recyclable/'
validation_data_dir = 'organic_and_recyclable/'

# amount of samples that will be used in training and validation sets
nb_train_samples = 400
nb_validation_samples = 100

epochs = 20
batch_size = 50

#### Clean data

Check if the image is in the right format
important to do this for every image neural network 
so, that the channels dont mix up, and that you dont get unexpected data

ref: https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/image/ImageDataGenerator
ref: https://www.codesofinterest.com/2017/09/keras-image-data-format.html

In [3]:
if K.image_data_format() == 'channels_first': #rgb
    input_shape = (3, img_width, img_height) # 3 for 3 layers
else:
    input_shape = (img_width, img_height, 3)

#### Generate training and tests sets

In [4]:
# preprocessing the training set
train_data_gen = ImageDataGenerator(rescale = 1. / 255, shear_range = 0.2, zoom_range = 0.2, horizontal_flip = True)

In [5]:
# preprocessing the test set
test_data_gen = ImageDataGenerator(rescale = 1. / 255)

In [6]:
# generate the data
train_generator = train_data_gen.flow_from_directory(
    train_data_dir, target_size=(img_width, img_height), color_mode="rgb", batch_size=batch_size, class_mode = 'binary')

Found 671 images belonging to 2 classes.


In [7]:
# generate the data
validation_generator = test_data_gen.flow_from_directory(
    validation_data_dir, target_size=(img_width, img_height), batch_size=batch_size, class_mode = 'binary')

Found 671 images belonging to 2 classes.


Below you can the 2 classes that is being mentioned above. Class O for Organic and Class R for Recyclable

In [8]:
print(train_generator.class_indices)

{'O': 0, 'R': 1}


#### Create neural network

Create neural network with Sequential

ref: https://www.ibm.com/cloud/learn/convolutional-neural-networks#toc-convolutio-JgBTyG9C
ref: https://towardsdatascience.com/a-comprehensive-guide-to-convolutional-neural-networks-the-eli5-way-3bd2b1164a53
ref: https://www.tensorflow.org/tutorials/images/classification
ref: https://keras.io/api/layers/convolution_layers/convolution2d/

In [9]:
# create Sequential object
model = Sequential()

# add to the Convolutional layer (extract features from the images)
# extract 32 features from the image, size of the search feature in pixels and iterate over all the pixels = 3, 3
model.add(Conv2D(32, (3, 3), input_shape=input_shape))

# Activation function is relu
model.add(Activation('relu'))

# ading our max pooling layer
model.add(MaxPooling2D(pool_size=(2, 2)))

# show what was the neural network has done
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 198, 198, 32)      896       
_________________________________________________________________
activation (Activation)      (None, 198, 198, 32)      0         
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 99, 99, 32)        0         
Total params: 896
Trainable params: 896
Non-trainable params: 0
_________________________________________________________________


ref: https://www.upgrad.com/blog/basic-cnn-architecture/
ref: https://www.geeksforgeeks.org/activation-functions-neural-networks/

In [10]:
# adding another convolutional layer and max pooling layer
model.add(Conv2D(32, (3, 3)))

# Activation function is relu
model.add(Activation('relu'))

# ading our max pooling layer
model.add(MaxPooling2D(pool_size=(2, 2)))

# adding another convolutional layer and max pooling layer
model.add(Conv2D(32, (3, 3)))

# Activation function is relu
model.add(Activation('relu'))

# ading our max pooling layer
model.add(MaxPooling2D(pool_size=(2, 2)))

# adding Our flattening Layer (flatten the image from 2d to 1d image)
model.add(Flatten())

# activate hidden layers which activate what the data is given then gives an output,
model.add(Dense(64))

# Activation function is relu
model.add(Activation('relu'))

# so it doesnt overfit, 
# if u want to make it faster so that it learns the images very well 
# then dont use dropouts
model.add(Dropout(0.5))

# add the output layer, one output, whether 0 or 1, sigmoid makes this possible
model.add(Dense(1))

# Activation function is sigmoid
model.add(Activation('sigmoid'))

# show what was the neural network has done
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 198, 198, 32)      896       
_________________________________________________________________
activation (Activation)      (None, 198, 198, 32)      0         
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 99, 99, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 97, 97, 32)        9248      
_________________________________________________________________
activation_1 (Activation)    (None, 97, 97, 32)        0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 48, 48, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 46, 46, 32)        9

Before training, first we compile then train the model using fit function

ref: https://nickmccullum.com/python-deep-learning/convolutional-neural-network-tutorial/#training-the-convolutional-neural-network
ref: https://docs.paperspace.com/machine-learning/wiki/accuracy-and-loss

In [14]:
# compile the model.
model.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy']) # rmsprop

# fit the generated model
model.fit(train_generator, steps_per_epoch=nb_train_samples // batch_size,
          epochs=epochs, validation_data=validation_generator,
          validation_steps=nb_validation_samples // batch_size)

Epoch 1/20
8/8 [==============================] - 16s 2s/step - loss: 0.7064 - accuracy: 0.8508 - val_loss: 0.1121 - val_accuracy: 0.9600
Epoch 2/20
8/8 [==============================] - 14s 2s/step - loss: 0.1937 - accuracy: 0.9341 - val_loss: 0.1573 - val_accuracy: 0.9300
Epoch 3/20
8/8 [==============================] - 14s 2s/step - loss: 0.1845 - accuracy: 0.9319 - val_loss: 0.1666 - val_accuracy: 0.9500
Epoch 4/20
8/8 [==============================] - 15s 2s/step - loss: 0.2320 - accuracy: 0.9032 - val_loss: 0.1034 - val_accuracy: 0.9400
Epoch 5/20
8/8 [==============================] - 14s 2s/step - loss: 0.1779 - accuracy: 0.9404 - val_loss: 0.0779 - val_accuracy: 0.9800
Epoch 6/20
8/8 [==============================] - 13s 2s/step - loss: 0.1877 - accuracy: 0.9240 - val_loss: 0.1375 - val_accuracy: 0.9500
Epoch 7/20
8/8 [==============================] - 14s 2s/step - loss: 0.1795 - accuracy: 0.9437 - val_loss: 0.1704 - val_accuracy: 0.9400
Epoch 8/20
8/8 [==================

We can see that the model is working fine. Validation loss started high then decreased, as for validation accuracy start low then increased. The model gives a validation accuracy of about 95%.

#### Save model

Always wise to do.

In [ ]:
model.save('classifier_three.h5')

# with open('cnn_model_95_acc.pkl', 'wb') as file:
#   pickle.dump(model, file)

#### Evaluate the model

After evaluating our model, we can see that our actual accuracy is 95.67% and our loss is around the 12.12

ref: https://stackoverflow.com/questions/34518656/how-to-interpret-loss-and-accuracy-for-a-machine-learning-model
ref: https://www.javacodemonk.com/difference-between-loss-accuracy-validation-loss-validation-accuracy-in-keras-ff358faa
ref: https://kharshit.github.io/blog/2018/12/07/loss-vs-accuracy

In [13]:
score = model.evaluate(validation_generator, verbose=1)

# print('loss=', score[0])
print('accuracy=', score[1])

14/14 [==============================] - 6s 424ms/step - loss: 0.2030 - accuracy: 0.9151
accuracy= 0.9150521755218506
